# NOTE: Scroll Down to County Report for instructions on use.
Just loading the PPP dataset will take considerable time.  Generating multiple reports in one session by editing and rerunning the blocks of code in the County Report section after running the whole notebook once is advised.  If storage space / generating time is an issue, individual components of the county report can be toggled on and off by manually editing the boolean values in the dictionary at the start of the county report section.

# Requirements
Please make sure that this notebook is saved at the same level as the script `datasets.py` as well as the folder named `perm_data` which contains the file `ppp2fips.csv` and `read_me_template.md`.  In addition, ensure all of the following libraries are installed before running this notebook.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import datasets
from urllib.request import urlopen
import json
import matplotlib.ticker as mtick
import os
import time
from shutil import copyfile


: 

# Data Loading/Prep
Before generating any reports we must load and process PPP data.  This will take much longer than generating the individual county reports.

## Loading

In [ ]:
# We will use the ppp function from the script datasets.py.
df= datasets.ppp()

In [ ]:
# We check the dataframe we have obtained (making sure that the info displays the number of non-null entries per column).
df.info(verbose=True, show_counts=True)
df

## NAICs Sector
Use [https://www.census.gov/naics/](https://www.census.gov/naics/) to look up individual NAICs codes.

In [4]:
# We create a new column from the first two characters of the NAICSCode.
df['NAICSSector']=df['NAICSCode'].astype(str).str.slice(stop=2)

In [ ]:
# Checking what the options are.  Note that 'na' was obtained by stripping the first two characters from NaN.
sorted(df['NAICSSector'].unique())

In [6]:
naics_sector_dict={
    '11': 'Agriculture, Forestry, Fishing and Hunting',
    '21': 'Mining, Quarrying, and Oil and Gas Extraction',
    '22': 'Utilities',
    '23': 'Construction',
    '31': 'Manufacturing',
    '32': 'Manufacturing',
    '33': 'Manufacturing',
    '42': 'Wholesale Trade',
    '44': 'Retail Trade',
    '45': 'Retail Trade',
    '48': 'Transportation and Warehousing',
    '49': 'Transportation and Warehousing',
    '51': 'Information',
    '52': 'Finance and Insurance',
    '53': 'Real Estate and Rental and Leasing',
    '54': 'Professional, Scientific, and Technical Services',
    '55': 'Management of Companies and Enterprises',
    '56': 'Administrative and Support and Waste Management and Remediation Services',
    '61': 'Educational Services',
    '62': 'Health Care and Social Assistance',
    '71': 'Arts, Entertainment, and Recreation',
    '72': 'Accommodation and Food Services',
    '81': 'Other Services (except Public Administration)',
    '92': 'Public Administration',
    '99': 'Nonclassifiable Establishments',
    'na':'Not Provided'
}

In [ ]:
# Now we remap our sector two digit codes to descriptions.
df['NAICSSector']=df['NAICSSector'].map(naics_sector_dict)
df[['NAICSCode','NAICSSector']]

## Jobs Reported Bins

In [8]:
# Let us also create a bin by the number of jobs reported.
df['JobsReportedBins']=pd.cut(df['JobsReported'],
                     [0,1,10,50,250,np.inf],
                     right=True,
                     include_lowest=True,
                     labels=['1','2-10','11-50','51-250','250+']
                     )

## FIPS Column
We want to include a column with FIPS values (which identify counties). We need to loan the CSV 
`ppp2fips.csv` in order to do so.

In [ ]:
# We load the converter (note that we need to load the FIPS code as a string not a float)
fips_converter=pd.read_csv('perm_data/ppp2fips.csv', dtype={'CountyFIPS':object})
fips_converter.info()
fips_converter

In [ ]:
# We merge the PPP data with this new dataframe based on the state and county names.
fips_converter.rename(columns={'ProjectState':'StateAbbr','ProjectCountyName':'CountyName'},inplace=True)
print(fips_converter)
df=df.merge(fips_converter,how='left',left_on=['ProjectState','ProjectCountyName'],right_on=['StateAbbr','CountyName'])
df

In [11]:
# Dropping duplicate columns
df.drop(columns=['StateAbbr','CountyName'], inplace=True)

In [ ]:
# Finally we check that our dataframe is only missing FIPS codes (when the county was given) outside of the 50 states.
df[(df['CountyFIPS']!=df['CountyFIPS'])&(df['ProjectCountyName']==df['ProjectCountyName'])]['ProjectState'].unique()

# Generating County Statistics
We will now create a dataframe with statistics for each county (rather than for each loan).  It will be called county_totals.

In [ ]:
county_totals=pd.DataFrame(df['CountyFIPS'].value_counts().rename('Number of Loans'))
county_totals.info()
county_totals

In [ ]:
county_totals=county_totals.merge(fips_converter,how='left',left_index=True,right_on='CountyFIPS')
county_totals

In [ ]:
# Because of how we put the converter together we get two entries here.  We will drop one.
print(county_totals[county_totals['CountyFIPS']=='02013'])
county_totals.info()
county_totals.drop_duplicates(subset=['CountyFIPS'],inplace=True)
print(county_totals[county_totals['CountyFIPS']=='02013'])
county_totals.info()

In [ ]:
# Cleaning up the dataframe:
county_totals.drop(columns=['NewCounties','STATE | COUNTY'], inplace=True)
county_totals.rename(columns={'StateAbbr':'State','CountyName':'County','CountyFIPS':'FIPS'},inplace=True)
county_totals.set_index('FIPS',inplace=True)
county_totals=county_totals[['State','County','Number of Loans']]
county_totals.sort_values(['State','County'],inplace=True)
county_totals.info()
county_totals

In [ ]:
# We add a column for the number of dollars approved per county.
temp_df=pd.DataFrame(df.groupby('CountyFIPS')['InitialApprovalAmount'].sum().rename('Dollars Approved'))
county_totals=pd.concat([county_totals,temp_df],axis=1)
county_totals.index.name='FIPS'
county_totals

In [ ]:
# Next for the number of jobs reported.
temp_df=pd.DataFrame(df.groupby('CountyFIPS')['JobsReported'].sum().rename('Jobs Reported'))
county_totals=pd.concat([county_totals,temp_df],axis=1)
county_totals.index.name='FIPS'
county_totals

In [ ]:
temp_df=pd.DataFrame(df[df['LoanStatus']=='Charged Off']['CountyFIPS'].value_counts().rename('Charge-Offs'))
county_totals=pd.concat([county_totals,temp_df],axis=1)
county_totals.index.name='FIPS'
# We assume that since we took value counts and sums any NaN values in our frame should represent 0s.  Hence we fillna appropriately.
county_totals.fillna(0,inplace=True)
county_totals

In [ ]:
# We get the charge-off rate of each county as the number of charge offs divided by the total number of loans.
county_totals['Charge-Off Rate']=county_totals['Charge-Offs']/county_totals['Number of Loans']
county_totals

# Plotly Map Prep
In order to use Plotly choropleth maps, we will need to load a json map.

In [21]:
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    county_map = json.load(response)

# If we want to Zoom in we will use the following.
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    selected_map = json.load(response)

In [22]:
# Depending on how we want to zoom in we may need to cut at the json map.  In order to do so we would need state numbers, not names.  
# The numbers are given here.
state_numbers_dictionary={
    'AK': '02',
    'AL': '01',
    'AZ': '04',
    'AR': '05',
    'CA': '06',
    'CO': '08',
    'CT': '09',
    'DE': '10',
    'DC': '11',
    'FL': '12',
    'GA': '13',
    'HI': '15',
    'ID': '16',
    'IL': '17',
    'IN': '18',
    'IA': '19',
    'KS': '20',
    'KY': '21',
    'LA': '22',
    'ME': '23',
    'MD': '24',
    'MA': '25',
    'MI': '26',
    'MN': '27',
    'MS': '28',
    'MO': '29',
    'MT': '30',
    'NE': '31',
    'NV': '32',
    'NH': '33',
    'NJ': '34',
    'NM': '35',
    'NY': '36',
    'NC': '37',
    'ND': '38',
    'OH': '39',
    'OK': '40',
    'OR': '41',  
    'PA': '42',
    'RI': '44',
    'SC': '45',
    'SD': '46',
    'TN': '47',
    'TX': '48',
    'UT': '49',
    'VT': '50',
    'VA': '51',
    'WA': '53',
    'WV': '54',
    'WI': '55',
    'WY': '56'
}

# National Statistics
We gather some national statistics on PPP loans before reaching the county report generating section of our code.

In [23]:
# The number of loans.
national_loans=df.shape[0]
# The number of dollars approved.
national_approved=df['InitialApprovalAmount'].sum()
# The number of loans charged off.
national_charge_offs=df[df['LoanStatus']=='Charged Off'].shape[0]
# The number of jobs reported.
national_jobs=df['JobsReported'].sum()

In [24]:
# We will eventually want statistics for demographics on the national, state, and county level. We can tally at the national level now
# to speed the computations once we start generating county statistics.
# First we collect the number of loans and loan dollars by race (cut by gender)
national_race=pd.concat([df.groupby(['Race','Gender']).size().rename('U.S. Loans'),
                         df.groupby(['Race','Gender'])['InitialApprovalAmount'].sum().rename('U.S. Loan Dollars')],axis=1)
# Next by gender.
national_gender=pd.concat([df['Gender'].value_counts().rename('U.S. Loans'),
                           df.groupby('Gender')['InitialApprovalAmount'].sum().rename('U.S. Loan Dollars')],axis=1)
# Then by ethnicity
national_ethnicity=pd.concat([df.groupby(['Ethnicity','Gender']).size().rename('U.S. Loans'),
                         df.groupby(['Ethnicity','Gender'])['InitialApprovalAmount'].sum().rename('U.S. Loan Dollars')],axis=1)
# By NAICs sector
national_naics_sector=pd.concat([df['NAICSSector'].value_counts().rename('U.S. Loans'),
                           df.groupby('NAICSSector')['InitialApprovalAmount'].sum().rename('U.S. Loan Dollars')],axis=1)
# And by the general number of jobs reported.  Since 'JobsReportedBins' is a category type, we have to pass observed=True to avoid a warning.
national_job_bins=pd.concat([df['JobsReportedBins'].value_counts().rename('U.S. Loans'),
                           df.groupby('JobsReportedBins',observed=True)['InitialApprovalAmount'].sum().rename('U.S. Loan Dollars')],axis=1)

# County Report
We can use the following code to generate a report for a given county.  Currently only states are supported (so counties in the Virgin Islands, Puerto Rico, etc cannot be selected). Use the three code blocks below to
1. Select which data/figures to save.
2. Find the FIPS code of the county you want to generate a report for.
3. Enter the desired FIPS code under `target_fips` in the third block, then execute cell and below.

## Choose Files to Save
In the following dictionary set a value to True if it should be included, or False if it should be omitted. Make sure to run the block once after editing the dictionary.

In [59]:
# Edit to true or false as appropriate
files_to_save={
    'counties map': True,
    'cities scatterplot': True,
    'county level boxplots': True,
    'loan level boxplots': True,
    'percent of state pie': True,
}


## Find Target FIPS
Use the block below to view FIPS codes and county names for a chosen state.

In [ ]:
print(county_totals['State'].unique())

# Run once to view the acceptable state abbreviations, then edit the below line to view the counties in the state you choose.
# To edit please change the value of check_state to check_state='YOUR_STATE_ABBR_HERE'.
# Take note of the FIPS code of the county you want to generate a report for.
check_state='MA'

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(county_totals[county_totals['State']==check_state]['County'])



## Choose Target FIPS Here
Set `target_fips=YOUR_FIPS_CODE` in the block below.  To obtain the county report you must execute the cell and below.

In [61]:
# Edit here then run this cell and all below it.
target_fips='25019'


# Leave this alone.
target_fips=str(target_fips)

## Folder Creation

In [ ]:
# The start time is recorded so that at the end of the county report generation we can see how long it took.
start=time.time()

# Using the selected target FIPS code we first obtain the target county and state names.
target_county=county_totals.loc[target_fips]['County']
target_state=county_totals.loc[target_fips]['State']
# We create a path to the folder where we will save the details of the report, of the form COUNTY_STATE.
file_path="reports/"+target_county.lower()+"_"+target_state.lower()+'_report/'

# We check to see if the reports folder exits.  If not, we create it. We do the same for the county report folder.
# Note we have to remove the last character, '/', when checking/creating the county report folder. 
if not os.path.exists('reports'):
    os.makedirs('reports')
if not os.path.exists(file_path[:-1]):
    os.makedirs(file_path[:-1])



## README Creation

In [ ]:
# We copy the read_me_template from permanent data, replace placeholder content with the county and state, 
# and save the result as README.md in the county report folder.

# First we capitalize the state correctly.
state_for_readme=target_state.upper()
# Next we load the county name.
county_for_readme=fips_converter.set_index('CountyFIPS').loc[target_fips]['NewCounties']




# Loading the template.
template_text=[line for line in open(r'perm_data\read_me_template.md')]
# Creating the new README
writer = open(file_path+"README.md",'w')
for line in template_text:
    line=line.replace('COUNTY_NAME', county_for_readme)
    line=line.replace('STATE_NAME', state_for_readme)
writer.close()

## Key Stats
We will assemble some key facts for the target county as well as for loans outside the county (nationally and in-state).

In [ ]:
key_facts=pd.DataFrame(county_totals.loc[target_fips].rename(target_county+', '+target_state).drop(['State','County']))
key_facts

In [64]:
# We will create a dataframe of all loans OUTSIDE the target county but in state.
outside_df=df[(df['ProjectState']==target_state)&(df['CountyFIPS']!=target_fips)]
# And a dataframe for all loans in the target county.
target_df=df[df['CountyFIPS']==target_fips]

In [ ]:
# We want to add facts about the national levels and the rest of the state (minus the target county).
# We will collect this information in one dataframe and then concatenate it with the key facts for the target county.

# We create an empty dataframe.
outside_facts=pd.DataFrame(index=['U.S.','Rest of State'])
outside_facts['Number of Loans']=[national_loans,
                                  outside_df.shape[0]]
outside_facts['Dollars Approved']=[national_approved,
                                   outside_df['InitialApprovalAmount'].sum()]
outside_facts['Jobs Reported']=[national_jobs,
                                   outside_df['JobsReported'].sum()]
outside_facts['Charge-Offs']=[national_charge_offs,
                                   outside_df[outside_df['LoanStatus']=='Charged Off'].shape[0]]
outside_facts['Charge-Off Rate']=outside_facts['Charge-Offs']/outside_facts['Number of Loans']
outside_facts

In [ ]:
# Let us combine these outside facts with our key facts!
# First we transpose the key facts so that the columns line up correctly.
key_facts=pd.concat([key_facts.T,outside_facts],axis=0)
# We will add some averages and rearrange columns
key_facts['Average Dollars Approved']=key_facts['Dollars Approved']/key_facts['Number of Loans']
key_facts['Average Jobs Reported']=key_facts['Jobs Reported']/key_facts['Number of Loans']
# We rearrange the columns and then transpose (so the order of the columns becomes the order of the rows)
key_facts=key_facts[[
    'Number of Loans',
    'Dollars Approved',
    'Average Dollars Approved',
    'Jobs Reported',
    'Average Jobs Reported',
    'Charge-Offs',
    'Charge-Off Rate']].T
# Finally, now that the location (state, national, target county) are the COLUMNS, we reorder THESE columns.
key_facts=key_facts[['{}, {}'.format(target_county,target_state),
                     'Rest of State',
                     'U.S.']]
key_facts

In [67]:
# We will store some totals for the target county, the rest of the state, and the target state in total.

# We choose to store these as independent variables as opposed to having to use .loc[] on our key_facts dataframe
# every time we need to access them.
target_county_loans=key_facts['{}, {}'.format(target_county,target_state)].loc['Number of Loans']
outside_loans=key_facts['Rest of State'].loc['Number of Loans']
target_state_loans=target_county_loans+outside_loans

target_county_loan_dollars=key_facts['{}, {}'.format(target_county,target_state)].loc['Dollars Approved']
outside_loan_dollars=key_facts['Rest of State'].loc['Dollars Approved']
target_state_loan_dollars=target_county_loan_dollars+outside_loan_dollars



### Percent of State

In [ ]:
# We will create a simple series consisting of what percent of the state total does the target county represent in a number of areas.

# First we select the areas to consider.
list_of_facts_to_compare=['Number of Loans','Dollars Approved','Jobs Reported','Charge-Offs']

# Then we collect the percent by dividing each fact by the sum of the county and the rest of the state (i.e. the state total)
percent_of_state=(key_facts.loc[list_of_facts_to_compare]['{}, {}'.format(target_county,target_state)]/(
    key_facts.loc[list_of_facts_to_compare]['{}, {}'.format(target_county,target_state)]+
    key_facts.loc[list_of_facts_to_compare]['Rest of State']
)).rename('Percent of State Totals')
percent_of_state

In [69]:
# We would like to graph these percents as pie charts as long
# as the target county represents enough of the state total to give a nice looking chart.
# We will use 1% as this minimum break point.  

# We only want to take time and space to do this if we have set the option to True in the dictionary at the start of the 
# county report generating section of this notebook.
if files_to_save['percent of state pie']:
    # rows_to_plot will store the index labels of those percentages we want to make pie charts of
    rows_to_plot=[]
    # We only include an index in rows_to_plot if the county is greater or equal to 1% of the state total in that stat.
    for stat in percent_of_state.index:
        if percent_of_state.loc[stat]>=.01:
            rows_to_plot.append(stat)
    rows_to_plot

In [70]:
# In this cell we display the pie charts of the stats saved in rows_to_plot.

sns.set_theme()
# We create a function which will make a pie chart
def plot_pie(stats,fig, axs):
    """Takes a list of indices (of the dataframe key_facts), a matplotlib figure, and a list of flattened axes,
    then produces a set of pie charts (one for each index).  Make sure the length of the lists stats and axs match."""
    # We want to iterate through the indices and the axes simultaneously, so we zip them together.
    for (stat,ax) in zip(stats,axs):
        # We create a pie chart at each axis of the target county versus the rest of the state (so the two together represent
        # the state total)
        ax.pie([key_facts['{}, {}'.format(target_county,target_state)].loc[stat],
            key_facts['Rest of State'].loc[stat]],
            labels=['{}, {}'.format(target_county,target_state),'Rest of State'],
            colors=sns.color_palette(), 
            autopct='%.2f%%',
            explode=[.01,0])
        ax.set_title(stat)

# We need different behavior depending on how long the list of indices is, and whether we want to save the figure at all.
# If we want to save the figure, we first consider if only one stat is greater than 1% of the state total.
if files_to_save['percent of state pie'] and len(rows_to_plot)==1:
    # Create a figure and a single axis.
    fig, axs = plt.subplots(1,1,figsize=(6,3))
    # We need to flatten the axes just in case, since plot_pie expects a list.
    axs=axs.flatten()
    fig.suptitle('Portion of State Total')
    plot_pie(rows_to_plot,fig=fig,axs=axs)
    plt.tight_layout()
    plt.savefig(file_path+'state_total_pie_chart.png')
    plt.show()
elif files_to_save['percent of state pie'] and len(rows_to_plot)>1:
    # If there is more than one index which is over 1%, we break the cases down further when creating the figure and axes.
    if len(rows_to_plot)==2:
        fig, axs = plt.subplots(1,2,figsize=(6,6))
    elif len(rows_to_plot)==3:
        fig, axs = plt.subplots(2,2,figsize=(12,6))
        # We only want three axes so we delete the lower right one.
        fig.delaxes(axs[1,1])
    elif len(rows_to_plot)==4:
        fig, axs = plt.subplots(2,2,figsize=(12,6))
    # In any case we now want to flatten the axes so we can call plot_pie()
    axs=axs.flatten()
    fig.suptitle('Portion of State Totals')
    plot_pie(rows_to_plot,fig=fig,axs=axs)
    plt.tight_layout()
    plt.savefig(file_path+'state_totals_pie_charts.png')
    plt.show()


### City Stats
We store some key facts on the cities inside the target county.

In [ ]:
# Gathering City Stats
# We want to combine the number of loans per city and the number of loan dollars per city.  We convert the cities to capital case to
# try and avoid counting loans separately if they were originally listed in different case styles.
city_df=pd.concat([
    target_df['ProjectCity'].str.upper().value_counts().rename('{} Loans'.format(target_county)),
    target_df.groupby(target_df['ProjectCity'].str.upper())['InitialApprovalAmount'].sum().rename('{} Loan Dollars'.format(target_county))
],axis=1)
# We now add what percent of the county total each city represents in these two areas.
city_df['{} Loan Percent'.format(target_county)]=(city_df['{} Loans'.format(target_county)]/target_county_loans)
city_df['{} Loan Dollars Percent'.format(target_county)]=(city_df['{} Loan Dollars'.format(target_county)]/target_county_loan_dollars)
city_df


In [ ]:
# Displaying city totals in a scatterplot.

# We only want to save the scatterplot of cities if the option has been checked at the start of this section.
if files_to_save['cities scatterplot']:
    # We will use a plotly scatterplot so that we can hover over datapoints for more information. On the x axis we will note the 
    # number of loans per city, and on the y axis the number of loan dollars (only loans and loan dollars listed in the county).
    fig=px.scatter(
        data_frame=city_df,
        x='{} Loans'.format(target_county),
        y='{} Loan Dollars'.format(target_county),
        title='Cities in {}, {}'.format(target_county,target_state),
        hover_name=city_df.index,
        template='seaborn',
        labels={ # For each source of data we have used, include 'source_name': 'Desired Name'
            '{} Loans'.format(target_county):'Loans',
            '{} Loan Dollars'.format(target_county): 'Loan Dollars',
    
        }
    )
    # We want to include dollar signs on the y values.
    fig.update_layout(yaxis_tickprefix = '$')
    # We save the figure appropriately.
    fig.write_html(file_path+'cities.html')
    fig.show()

### Counties Map
We create a choropleth map of the target state so we can compare county facts (with the target county highlighted).

In [ ]:
if files_to_save['counties map']:
    # We need to obtain the state number from our dictionary for the target state and make sure our selected map only has features
    # for that target state.
    ####################
    selected_state_number=state_numbers_dictionary[target_state]
    selected_map['features'] = [f for f in county_map['features'] if f['properties']['STATE'] == selected_state_number]
    ####################
    # We select a dataframe to make a choropleth from.  We want county information in the target state, so we cut the county_totals
    # frame to include only those counties in the target state.
    graphable_df=county_totals[county_totals['State']==target_state]

    # We now create the choropleth.
    fig=px.choropleth(graphable_df,
                    geojson=selected_map,
                    locations=graphable_df.index, # Recall county_totals is indexed by FIPS codes (originally a string before becoming the index).
                    color='Dollars Approved', # We will let the number of loan dollars determine the color of each county.
                    hover_name='County', # The header of the hover text will be the county name.
                    hover_data=['Number of Loans'], # The additional data we want in the hoverbox.
                    color_continuous_scale="Viridis",
                    scope='usa',
                    labels={ # For each source of data you have used, include 'source_name': 'Desired Name'.
            'Number of Loans':'Loans',
            'Dollars Approved': 'Loan Dollars',
        })
    # The following sets the plot to zoom in on our target state.
    fig.update_geos(fitbounds="locations",visible=False)
    # We add borders around the state in black.  
    state_borders = go.Choropleth(
            locationmode='USA-states',
            z=[0,0,0,0,0,0,0,0,0,0,0,0],
            locations=[target_state],
            colorscale = [[0,'rgba(0, 0, 0, 0)'],[1,'rgba(0, 0, 0, 0)']],
            marker_line_color='Black',
            showscale = False,
            hoverinfo='skip' # We do not want to block or edit our present hover information.
        )
    # We also want a border around the chosen county to make it obvious.
    county_border= go.Choropleth(
            geojson=selected_map,
            z=[0,0,0,0,0,0,0,0,0,0,0,0],
            locations=[target_fips],
            colorscale = [[0,'rgba(0, 0, 0, 0)'],[1,'rgba(0, 0, 0, 0)']],
            marker_line_color='red',
            showscale = False,
            hoverinfo='skip'
        )

    # When creating this it is useful to view the hover template and then update traces.
    # Checking the hover template.
    #print(fig.data[0].hovertemplate)

    # Adjusting the template to reorder and remove the FIPS code.
    fig.update_traces(hovertemplate='<b>%{hovertext}</b><br>Loans: %{customdata[0]} <br>Loan Dollars: $%{z}') 

    fig.add_trace(state_borders)
    fig.add_trace(county_border)
    fig.write_html(file_path+"counties_map.html")
    fig.show()

## Demographic Stats
In this section we store information on various demographic stats (gender, race, ethnicity).

In [74]:
# We will create a function to add percentage columns to a given dataframe.
# We will add columns representing what percent of the respective total each row represents.
# For example, given the gender dataframe, the 'Male Owned' row of the U.S. Loan Percent gives the percent of all loans in the U.S. 
# which were given as male owned. Likewise the 'Male Owned' row of Rest of State Loan Dollars Percent gives the percent of all loan dollars in 
# the target state (but not the target county) which went to male owned businesses.
def add_percent_columns(dataframe):
    """Accepts a dataframe with target_county, Rest of State, and U.S. Loans and Loan Dollars as columns, then creates additional columns
    representing what percent of the U.S. total each stat represents.  Two columns representing the different in percent between the
    target county and the rest of the state are also added."""
    dataframe['{} Loan Percent'.format(target_county)]=(dataframe['{} Loans'.format(target_county)]/target_county_loans)
    dataframe['{} Loan Dollars Percent'.format(target_county)]=(dataframe['{} Loan Dollars'.format(target_county)]/target_county_loan_dollars)
    dataframe['Rest of State Loan Percent']=(dataframe['Rest of State Loans']/outside_loans)
    dataframe['Rest of State Loan Dollars Percent']=(dataframe['Rest of State Loan Dollars']/outside_loan_dollars)
    dataframe['U.S. Loan Percent']=(dataframe['U.S. Loans']/national_loans)
    dataframe['U.S. Loan Dollars Percent']=(dataframe['U.S. Loan Dollars']/national_approved)
    # We will add columns to locate outliers from the rest of the state.
    dataframe['Loan Outliers']=abs(dataframe['{} Loan Percent'.format(target_county)]-dataframe['Rest of State Loan Percent'])
    dataframe['Loan Dollars Outliers']=abs(dataframe['{} Loan Dollars Percent'.format(target_county)]-dataframe['Rest of State Loan Dollars Percent'])
    return dataframe
    

In [ ]:
# Gathering Gender stats
gender_df=pd.concat([
    national_gender,
    target_df['Gender'].value_counts().rename('{} Loans'.format(target_county)),
    target_df.groupby('Gender')['InitialApprovalAmount'].sum().rename('{} Loan Dollars'.format(target_county)),
    outside_df['Gender'].value_counts().rename('Rest of State Loans'),
    outside_df.groupby('Gender')['InitialApprovalAmount'].sum().rename('Rest of State Loan Dollars')    
],axis='columns')

# We fill all nan values with 0 then apply our custom function.
gender_df.fillna(0,inplace=True)
gender_df=add_percent_columns(gender_df)
gender_df.sort_index(inplace=True)
gender_df

In [ ]:
# Gathering statistics by race.  Here we want a multi-index.
race_df=pd.concat([
    national_race,
    target_df.groupby(['Race','Gender']).size().rename('{} Loans'.format(target_county)),
    target_df.groupby(['Race','Gender'])['InitialApprovalAmount'].sum().rename('{} Loan Dollars'.format(target_county)),
    outside_df.groupby(['Race','Gender']).size().rename('Rest of State Loans'),
    outside_df.groupby(['Race','Gender'])['InitialApprovalAmount'].sum().rename('Rest of State Loan Dollars'),
],axis='columns')
# We fill blanks with 0
race_df.fillna(0,inplace=True)

# We would like to have totals for each race. 
# Each index in race_df.index is a list of two elements (the first and second level label).
# We want to iterate through the first level indices
for index in {i[0] for i in race_df.index}:
    # A warning is thrown after a certain number of iterations if we don't sort here.
    race_df.sort_index(inplace=True)
    # We want to create a second level label for each first level label representing the _TOTAL_ across all genders.
    race_df.loc[(index,'_TOTAL_'),:]=race_df.loc[(index,),:].sum()

# We can now sort the index completely (the underscore before TOTAL will make sure it comes last among the second level labels)
race_df.sort_index(inplace=True)    
# With the correct number of rows we can now add the additional columns.
race_df=add_percent_columns(race_df)
race_df


In [ ]:
# Gathering ethnicity stats.  
# Again, we want to use a multi-index.  We use the same approach as the previous cell (see above for details).
ethnicity_df=pd.concat([
    national_ethnicity,
    target_df.groupby(['Ethnicity','Gender']).size().rename('{} Loans'.format(target_county)),
    target_df.groupby(['Ethnicity','Gender'])['InitialApprovalAmount'].sum().rename('{} Loan Dollars'.format(target_county)),
    outside_df.groupby(['Ethnicity','Gender']).size().rename('Rest of State Loans'),
    outside_df.groupby(['Ethnicity','Gender'])['InitialApprovalAmount'].sum().rename('Rest of State Loan Dollars'),
],axis='columns')
ethnicity_df.fillna(0,inplace=True)

for index in {i[0] for i in ethnicity_df.index}:
    ethnicity_df.sort_index(inplace=True)
    ethnicity_df.loc[(index,'_TOTAL_'),:]=ethnicity_df.loc[(index,),:].sum()
ethnicity_df.sort_index(inplace=True)
ethnicity_df=add_percent_columns(ethnicity_df)
ethnicity_df

## NAICs/Business Stats
We collect stats for business sizes, NAICs sectors, and NAICs codes.

In [ ]:
# Job bins
job_bins_df=pd.concat([
    national_job_bins,
    target_df['JobsReportedBins'].value_counts().rename('{} Loans'.format(target_county)),
    target_df.groupby('JobsReportedBins',observed=True)['InitialApprovalAmount'].sum().rename('{} Loan Dollars'.format(target_county)),
    outside_df['JobsReportedBins'].value_counts().rename('Rest of State Loans'),
    outside_df.groupby('JobsReportedBins',observed=True)['InitialApprovalAmount'].sum().rename('Rest of State Loan Dollars')    
],axis='columns')
job_bins_df.fillna(0,inplace=True)
job_bins_df=add_percent_columns(job_bins_df)
job_bins_df

In [ ]:
# Naics Sector
naics_sector_df=pd.concat([
    national_naics_sector, 
    target_df['NAICSSector'].value_counts().rename('{} Loans'.format(target_county)),
    target_df.groupby('NAICSSector')['InitialApprovalAmount'].sum().rename('{} Loan Dollars'.format(target_county)),
    outside_df['NAICSSector'].value_counts().rename('Rest of State Loans'),
    outside_df.groupby('NAICSSector')['InitialApprovalAmount'].sum().rename('Rest of State Loan Dollars')    
],axis='columns')

# We fill all nan values with 0 then apply our custom function.
naics_sector_df.fillna(0,inplace=True)
naics_sector_df=add_percent_columns(naics_sector_df)
naics_sector_df.sort_index(inplace=True)
naics_sector_df

In [ ]:
# Naics Code (no national values needed.  We will only use this for outliers)
naics_code_df=pd.concat([
    target_df['NAICSCode'].value_counts().rename('{} Loans'.format(target_county)),
    target_df.groupby('NAICSCode')['InitialApprovalAmount'].sum().rename('{} Loan Dollars'.format(target_county)),
    outside_df['NAICSCode'].value_counts().rename('Rest of State Loans'),
    outside_df.groupby('NAICSCode')['InitialApprovalAmount'].sum().rename('Rest of State Loan Dollars')    
],axis='columns')

# We fill all nan values with 0 then continue
naics_code_df.fillna(0,inplace=True)

# Because this dataframe does not have U.S. level information, we will have to manually put together the same kinds of columns as in
# the function add_percent_columns().
naics_code_df['{} Loan Percent'.format(target_county)]=(naics_code_df['{} Loans'.format(target_county)]/target_county_loans)
naics_code_df['{} Loan Dollars Percent'.format(target_county)]=(naics_code_df['{} Loan Dollars'.format(target_county)]/target_county_loan_dollars)
naics_code_df['Rest of State Loan Percent']=(naics_code_df['Rest of State Loans']/outside_loans)
naics_code_df['Rest of State Loan Dollars Percent']=(naics_code_df['Rest of State Loan Dollars']/outside_loan_dollars)
# We will add column to locate outliers from the rest of the state.
naics_code_df['Loan Outliers']=abs(naics_code_df['{} Loan Percent'.format(target_county)]-naics_code_df['Rest of State Loan Percent'])
naics_code_df['Loan Dollars Outliers']=abs(naics_code_df['{} Loan Dollars Percent'.format(target_county)]-naics_code_df['Rest of State Loan Dollars Percent'])
naics_code_df.index=[str(i).split('.', 1)[0] for i in naics_code_df.index]
naics_code_df

## Descriptive Stats
We will prepare descriptive stats (min, max, quartiles, standard deviation, etc) at two levels.  First counting the totals for each county as the instances, and then by using each individual loan.

### County Level
We would like to put together boxplots for the different county level descriptive stats.  Using Seaborn it is often easiest to create multiple boxes by making each box represent a column in the same dataframe.  In particular our goal will be to have two boxes per plot-- one to represent the descriptive stats when all counties in the U.S. are considered, and another restricted to the target state.  The set of counties in the target state is obviously smaller (and will take less rows) but we populate the other rows with NaN values.  Seaborn will ignore NaN entries when building the boxplots.

In [ ]:
# We need to have a dataframe with every county in the U.S. as rows, but then additional columns in which the only rows with non-null entries 
# lie in the target state.
in_state_columns = [county_totals[county_totals['State']==target_state][column].rename('In-State '+column)
                     for column in county_totals.drop(columns=['State','County']).columns]
county_level_boxplots=pd.concat([county_totals]+in_state_columns,axis=1)
county_level_boxplots

In [82]:
# Graphing will look better if we use thousands of dollars as our measurement.
county_level_boxplots['Dollars Approved (in thousands)']=county_level_boxplots['Dollars Approved']/1000
county_level_boxplots['In-State Dollars Approved (in thousands)']=county_level_boxplots['In-State Dollars Approved']/1000

In [ ]:
# We will only create and save the county level boxplots if the option was chosen earlier.
if files_to_save['county level boxplots']:
    sns.set_theme()
    # We want four plots in a two by two grid.
    fig, axs = plt.subplots(2,2,figsize=(12,10))
    axs=axs.flatten()
    # The first plot represents the number of loans per county (in the U.S. versus in the target state)
    sns.boxplot(data=county_level_boxplots[['Number of Loans','In-State Number of Loans']].rename(
        columns={'Number of Loans':'National','In-State Number of Loans':'In-State'}),
        showfliers=False, # We do not show fliers, so there could be large outliers.
        ax=axs[0])
    plt.suptitle('Statistics by County')
    # We will add a horizontal line indicating the number of loans in the target county.
    axs[0].axhline(y=county_totals.loc[target_fips]['Number of Loans'],
                xmin=0,
                xmax=.75,
                color=sns.color_palette()[3],
                lw=2,
                ls='--') 
    # We will also annotate the line appropriately.  
    text=axs[0].text(.75, county_totals.loc[target_fips]['Number of Loans'], 
            " "+county_totals.loc[target_fips]['County']+" ({0:,.0f})".format(county_totals.loc[target_fips]['Number of Loans']), 
            ha='left', 
            va='center',
            color=sns.color_palette()[3],
            fontsize=8,
            weight='bold',
            transform=axs[0].get_yaxis_transform())
    # We set a bbox for the annotation.
    text.set_bbox(dict(facecolor='white', edgecolor='black', boxstyle='round', alpha=.8))
    axs[0].title.set_text('Number of Loans')
    # Finally we format the y axis.
    fmt = '{x:,.0f}'
    tick = mtick.StrMethodFormatter(fmt)
    axs[0].yaxis.set_major_formatter(tick) 


    # We repeat the process for the dollars approved, jobs reported, and charge-off rate.  The key difference is changing the y-axis format.
    sns.boxplot(data=county_level_boxplots[['Dollars Approved (in thousands)','In-State Dollars Approved (in thousands)']].rename(
        columns={'Dollars Approved (in thousands)':'National','In-State Dollars Approved (in thousands)':'In-State'}),
        showfliers=False,
        ax=axs[1])

    axs[1].axhline(y=county_totals.loc[target_fips]['Dollars Approved']/1000,
                xmin=0,
                xmax=.75,
                color=sns.color_palette()[3],
                lw=2,
                ls='--') 
    text=axs[1].text(.75, county_totals.loc[target_fips]['Dollars Approved']/1000, 
            " "+county_totals.loc[target_fips]['County']+" (${0:,.0f}K)".format(round(county_totals.loc[target_fips]['Dollars Approved']/1000)), 
            ha='left', 
            va='center',
            color=sns.color_palette()[3],
            fontsize=8,
            weight='bold',
            transform=axs[1].get_yaxis_transform())
    text.set_bbox(dict(facecolor='white', edgecolor='black', boxstyle='round', alpha=.8))
    axs[1].title.set_text('Dollars Approved (in thousands)')
    fmt = '${x:,.0f}K'
    tick = mtick.StrMethodFormatter(fmt)
    axs[1].yaxis.set_major_formatter(tick) 



    sns.boxplot(data=county_level_boxplots[['Jobs Reported','In-State Jobs Reported']].rename(
        columns={'Jobs Reported':'National','In-State Jobs Reported':'In-State'}),
        showfliers=False,
        ax=axs[2])

    axs[2].axhline(y=county_totals.loc[target_fips]['Jobs Reported'],
                xmin=0,
                xmax=.75,
                color=sns.color_palette()[3],
                lw=2,
                ls='--') 
    text=axs[2].text(.75, county_totals.loc[target_fips]['Jobs Reported'], 
            " "+county_totals.loc[target_fips]['County']+" ({0:,.0f})".format(county_totals.loc[target_fips]['Jobs Reported']), 
            ha='left', 
            va='center',
            color=sns.color_palette()[3],
            fontsize=8,
            weight='bold',
            transform=axs[2].get_yaxis_transform())
    text.set_bbox(dict(facecolor='white', edgecolor='black', boxstyle='round', alpha=.8))
    axs[2].title.set_text('Jobs Reported')
    fmt = '{x:,.0f}'
    tick = mtick.StrMethodFormatter(fmt)
    axs[2].yaxis.set_major_formatter(tick) 


    sns.boxplot(data=county_level_boxplots[['Charge-Off Rate','In-State Charge-Off Rate']].rename(
        columns={'Charge-Off Rate':'National','In-State Charge-Off Rate':'In-State'}),
        showfliers=False,
        ax=axs[3])

    axs[3].axhline(y=county_totals.loc[target_fips]['Charge-Off Rate'],
                xmin=0,
                xmax=.75,
                color=sns.color_palette()[3],
                lw=2,
                ls='--') 
    text=axs[3].text(.75, county_totals.loc[target_fips]['Charge-Off Rate'], 
            " "+county_totals.loc[target_fips]['County']+" ({:.1%})".format(county_totals.loc[target_fips]['Charge-Off Rate']), 
            ha='left', 
            va='center',
            color=sns.color_palette()[3],
            fontsize=8,
            weight='bold',
            transform=axs[3].get_yaxis_transform())
    text.set_bbox(dict(facecolor='white', edgecolor='black', boxstyle='round', alpha=.8))
    axs[3].title.set_text('Charge-Off Rate')
    axs[3].yaxis.set_major_formatter(mtick.PercentFormatter(1)) 

    plt.tight_layout()
    plt.savefig(file_path+'county_level_boxplots.png')
    plt.show()

### Loan Level
At the loan level using Seaborn right away can take too long.  Instead we will strip the key stats from pandas.describe() and feed the quartiles into a Seaborn boxplot.  We will consider the number of dollars approved and the number of jobs per loan in the U.S., in the target state, and in the target county.

In [ ]:
loan_level_dollars=pd.concat([
    df['InitialApprovalAmount'].describe().rename('National'),
    df[df['ProjectState']==target_state]['InitialApprovalAmount'].describe().rename('In-State'),
    target_df['InitialApprovalAmount'].describe().rename(target_county+', '+target_state),
], axis=1)
loan_level_dollars

In [ ]:
loan_level_jobs=pd.concat([
    df['JobsReported'].describe().rename('National'),
    df[df['ProjectState']==target_state]['JobsReported'].describe().rename('In-State'),
    target_df['JobsReported'].describe().rename(target_county+', '+target_state),
], axis=1)
loan_level_jobs

In [ ]:
# When creating our boxplots we want to match the format of a typical Seaborn plot which has whiskers which extend to 
# the largest value in the data which is within 1.5 times the interquartile range (the 75% percentile minus the 25%)
# of the 75% percentile.
loan_level_dollar_boxes=pd.DataFrame()
for column in loan_level_dollars.columns:
    iqr=(loan_level_dollars.loc['75%'][column]-loan_level_dollars.loc['25%'][column])
    loan_level_dollar_boxes[column]=(
        loan_level_dollars.iloc[4:7][column].to_list()+ # iloc[4:7] gives the quartiles (25%, 50%, 75%)
        [min(loan_level_dollars.loc['max'][column],1.5*iqr+loan_level_dollars.loc['75%'][column]-1)]+ # We subtract 1 to make sure the result in range of Seaborn
        [max(loan_level_dollars.loc['min'][column],loan_level_dollars.loc['25%'][column]+1-1.5*iqr,0)]
    )

loan_level_dollar_boxes=loan_level_dollar_boxes/1000
loan_level_dollar_boxes


In [ ]:
loan_level_job_boxes=pd.DataFrame()
for column in loan_level_jobs.columns:
    iqr=(loan_level_jobs.loc['75%'][column]-loan_level_jobs.loc['25%'][column])
    loan_level_job_boxes[column]=(
        loan_level_jobs.iloc[4:7][column].to_list()+
        [min(loan_level_jobs.loc['max'][column],1.5*iqr+loan_level_jobs.loc['75%'][column])]+
        [max(loan_level_jobs.loc['min'][column],loan_level_jobs.loc['25%'][column]+-1.5*iqr,0)]
    )

loan_level_job_boxes

In [ ]:
# Creating and saving loan level boxplots if the option was checked.
if files_to_save['loan level boxplots']:
    sns.set_theme()
    fig, axs = plt.subplots(1,2,figsize=(10,6))
    axs=axs.flatten()
    sns.boxplot(data=loan_level_dollar_boxes,
                showfliers=False,
                palette=[sns.color_palette()[0],sns.color_palette()[1],sns.color_palette()[3]],
                ax=axs[0])
    axs[0].title.set_text('Dollars Approved (in thousands)')
    fmt = '${x:,.0f}K'
    tick = mtick.StrMethodFormatter(fmt)
    axs[0].yaxis.set_major_formatter(tick)

    sns.boxplot(data=loan_level_job_boxes,
                showfliers=False,
                palette=[sns.color_palette()[0],sns.color_palette()[1],sns.color_palette()[3]],
                ax=axs[1])

    axs[1].title.set_text('Jobs Reported')
    fmt = '{x:,.0f}'
    tick = mtick.StrMethodFormatter(fmt)
    axs[1].yaxis.set_major_formatter(tick) 
    plt.suptitle('Loan Level Statistics')
    plt.tight_layout()
    plt.savefig(file_path+'loan_level_boxplots.png')
    plt.show()

## Outliers Overview
We will create a dataframe consisting of the largest outliers for each demographic and business type based on loans,then one based on loan dollars.

In [ ]:
# On loans
# We add a function which adds dictionaries to a list, where each dictionary represents a row in a dataframe we intend to make.
def add_loan_outliers(list,dataframe,category):
    """Takes a list, dataframe, and a category; then for each row of the top 10 rows (with respect to the loan outlier feature)
    adds a dictionary to the list of the form column_name:row_entry."""
    for index, row in dataframe.nlargest(n=10,columns='Loan Outliers').iterrows():
        list.append({
            'Outlier':row.name,
            'Category':category,
            '{} Loans'.format(target_county):row['{} Loans'.format(target_county)],
            '{} Loan Percent'.format(target_county):row['{} Loan Percent'.format(target_county)],
            'Rest of State Loans':row['Rest of State Loans'],
            'Rest of State Loan Percent':row['Rest of State Loan Percent'],
            '{} Loan Dollars'.format(target_county):row['{} Loan Dollars'.format(target_county)],
            '{} Loan Dollars Percent'.format(target_county):row['{} Loan Dollars Percent'.format(target_county)],
            'Rest of State Loan Dollars':row['Rest of State Loan Dollars'],
            'Rest of State Loan Dollars Percent':row['Rest of State Loan Dollars Percent'],
            'Loan Outliers':row['Loan Outliers']
            })
# We start a list.
outlier_loans_list=[]
# For various dataframes we have made we now add the top outliers to our list
add_loan_outliers(outlier_loans_list,gender_df.drop('Unanswered',axis=0),'Gender') # We won't count outliers for Unanswered.
add_loan_outliers(outlier_loans_list,race_df.drop('Unanswered',axis=0,level=0).drop('Unanswered',axis=0,level=1),'Race/Gender')
add_loan_outliers(outlier_loans_list,ethnicity_df.drop('Unknown/NotStated',axis=0,level=0).drop('Unanswered',axis=0,level=1),'Ethnicity/Gender')
add_loan_outliers(outlier_loans_list,job_bins_df,'Jobs Reported')
add_loan_outliers(outlier_loans_list,naics_sector_df,'NAICs Sector')
add_loan_outliers(outlier_loans_list,naics_code_df,'NAICs Code')
# We make a dataframe from this list.
outlier_loans_df=pd.DataFrame(outlier_loans_list)
outlier_loans_df.set_index('Outlier',inplace=True)
outlier_loans_df.sort_values(by='Loan Outliers', ascending=False,inplace=True)
outlier_loans_df

In [ ]:
# On loan dollars:
# Repeating the technique from the previous cell, this time for loan dollar outliers.
def add_dollar_outliers(list,dataframe,category):
    for index, row in dataframe.nlargest(n=10,columns='Loan Dollars Outliers').iterrows():
        list.append({
            'Outlier':row.name,
            'Category':category,
            '{} Loans'.format(target_county):row['{} Loans'.format(target_county)],
            '{} Loan Percent'.format(target_county):row['{} Loan Percent'.format(target_county)],
            'Rest of State Loans':row['Rest of State Loans'],
            'Rest of State Loan Percent':row['Rest of State Loan Percent'],
            '{} Loan Dollars'.format(target_county):row['{} Loan Dollars'.format(target_county)],
            '{} Loan Dollars Percent'.format(target_county):row['{} Loan Dollars Percent'.format(target_county)],
            'Rest of State Loan Dollars':row['Rest of State Loan Dollars'],
            'Rest of State Loan Dollars Percent':row['Rest of State Loan Dollars Percent'],
            'Loan Dollars Outliers':row['Loan Dollars Outliers']
            })

outlier_dollars_list=[]
add_dollar_outliers(outlier_dollars_list,gender_df.drop('Unanswered',axis=0),'Gender')
add_dollar_outliers(outlier_dollars_list,race_df.drop('Unanswered',axis=0,level=0).drop('Unanswered',axis=0,level=1),'Race/Gender')
add_dollar_outliers(outlier_dollars_list,ethnicity_df.drop('Unknown/NotStated',axis=0,level=0).drop('Unanswered',axis=0,level=1),'Ethnicity/Gender')
add_dollar_outliers(outlier_dollars_list,job_bins_df,'Jobs Reported')
add_dollar_outliers(outlier_dollars_list,naics_sector_df,'NAICs Sector')
add_dollar_outliers(outlier_dollars_list,naics_code_df,'NAICs Code')
outlier_dollars_df=pd.DataFrame(outlier_dollars_list)
outlier_dollars_df.set_index('Outlier',inplace=True)
outlier_dollars_df.sort_values(by='Loan Dollars Outliers', ascending=False,inplace=True)
outlier_dollars_df

## Saving The Excel Report

In [ ]:
# For the demographic and business data we want to save only some of the columns.  We will store the following
columns_to_save=['{} Loans'.format(target_county),
                 '{} Loan Percent'.format(target_county),
                 '{} Loan Dollars'.format(target_county),
                 '{} Loan Dollars Percent'.format(target_county),
                 '{} Loans'.format('Rest of State'),
                 '{} Loan Percent'.format('Rest of State'),
                 '{} Loan Dollars'.format('Rest of State'),
                 '{} Loan Dollars Percent'.format('Rest of State'),
                 '{} Loans'.format('U.S.'),
                 '{} Loan Percent'.format('U.S.'),
                 '{} Loan Dollars'.format('U.S.'),
                 '{} Loan Dollars Percent'.format('U.S.')
        ]
columns_to_save

In [92]:
# We conclude by saving various statistics to an excel file.
# Creating the file.
writer= pd.ExcelWriter(file_path+'county_report.xlsx', engine='xlsxwriter')
# Creating the formats we will use.
header_format = writer.book.add_format({'bold':True,'text_wrap': True})
number_format = writer.book.add_format({'num_format': '#,##'})
currency_format= writer.book.add_format({'num_format':'$#,##0.00'})
percent_format = writer.book.add_format({'num_format':'0.00%'})
wrap_format=writer.book.add_format({'text_wrap': True})


########################################################################################################
# Overview:
########################################################################################################
# We will create a sheet labeled 'Overview' which gives general details on the target county using information we gather in the
# key facts section of this notebook.
key_facts.to_excel(writer, sheet_name='Overview', startrow=0, startcol=0)
# We will convert all to number format then adjust the currency and percentage fields afterwards.
writer.sheets['Overview'].set_column('A:D',20,number_format)
writer.sheets['Overview'].set_column('A:A',25, number_format)
writer.sheets['Overview'].set_column('C:C',25, number_format)
writer.sheets['Overview'].set_column('E:E',20, percent_format)
writer.sheets['Overview'].conditional_format(2, 0, 3, 3, {'type':'no_blanks','format':currency_format})
writer.sheets['Overview'].conditional_format(7, 0, 7, 3, {'type':'no_blanks','format':percent_format})

percent_of_state.to_excel(writer, sheet_name='Overview', startrow=10, startcol=0)
writer.sheets['Overview'].conditional_format(11, 1, 14, 1, {'type':'no_blanks','format':percent_format})

# Adding top cities information
city_df.nlargest(n=3,columns='{} Loans'.format(target_county))[[
        '{} Loans'.format(target_county),
        '{} Loan Percent'.format(target_county),
        '{} Loan Dollars'.format(target_county),
        '{} Loan Dollars Percent'.format(target_county),
        ]].to_excel(writer, sheet_name='Overview', startrow=18, startcol=0)
writer.sheets['Overview'].write_string(17, 0, 'Top 3 Cities (by most loans)', header_format)
writer.sheets['Overview'].write_string(18, 0, 'Project City', header_format)
city_df.nlargest(n=3,columns='{} Loan Dollars'.format(target_county))[[
        '{} Loans'.format(target_county),
        '{} Loan Percent'.format(target_county),
        '{} Loan Dollars'.format(target_county),
        '{} Loan Dollars Percent'.format(target_county),
        ]].to_excel(writer, sheet_name='Overview', startrow=25, startcol=0)
writer.sheets['Overview'].write_string(24, 0, 'Top 3 Cities (by most loan dollars)', header_format)
writer.sheets['Overview'].write_string(25, 0, 'Project City', header_format)
# Now we format this information correctly.
writer.sheets['Overview'].conditional_format(19, 2, 29, 2, {'type':'no_blanks','format':percent_format})
writer.sheets['Overview'].conditional_format(19, 3, 29, 3, {'type':'no_blanks','format':currency_format})
writer.sheets['Overview'].conditional_format(19, 4, 29, 4, {'type':'no_blanks','format':percent_format})


########################################################################################################
# Outliers Overview:
########################################################################################################
outlier_loans_df.nlargest(n=5,columns='Loan Outliers').drop(columns='Loan Outliers').to_excel(
    writer, sheet_name='Outliers (at a glance)', startrow=1, startcol=0)
writer.sheets['Outliers (at a glance)'].write_string(0, 0, 'Top 5 Outliers (by loans)', header_format) 

outlier_dollars_df.nlargest(n=5,columns='Loan Dollars Outliers').drop(columns='Loan Dollars Outliers').to_excel(
    writer, sheet_name='Outliers (at a glance)', startrow=10, startcol=0)
writer.sheets['Outliers (at a glance)'].write_string(9, 0, 'Top 5 Outliers (by loan dollars)', header_format) 
writer.sheets['Outliers (at a glance)'].set_column('A:J',20,number_format)
writer.sheets['Outliers (at a glance)'].set_column('A:A',25)
writer.sheets['Outliers (at a glance)'].set_column('B:B',15)
writer.sheets['Outliers (at a glance)'].set_column('D:D',20,percent_format)
writer.sheets['Outliers (at a glance)'].set_column('F:F',20,percent_format)
writer.sheets['Outliers (at a glance)'].set_column('G:G',20,currency_format)
writer.sheets['Outliers (at a glance)'].set_column('H:H',20,percent_format)
writer.sheets['Outliers (at a glance)'].set_column('I:I',20,currency_format)
writer.sheets['Outliers (at a glance)'].set_column('J:J',20,percent_format)

########################################################################################################
# Descriptive stats:
########################################################################################################
# We will create a sheet labeled 'County Level Descriptive Stats' and move on to a sheet labeled 'Loan Level Descriptive Stats'
pd.concat([county_totals['Number of Loans'].describe().rename('National'),
            county_totals[county_totals['State']==target_state]['Number of Loans'].describe().rename('In-State')],
            axis=1).to_excel(writer, sheet_name='County Level Descriptive Stats', startrow=0, startcol=0)
# Now we format these cells
writer.sheets['County Level Descriptive Stats'].conditional_format(0, 1, 9, 2, {'type':'no_blanks','format':number_format})
# And finally add a header for this set of descriptive stats.
writer.sheets['County Level Descriptive Stats'].write_string(0, 0, 'Number of Loans', header_format)
# We repeat the process four times.
pd.concat([county_totals['Dollars Approved'].describe().rename('National'),
            county_totals[county_totals['State']==target_state]['Dollars Approved'].describe().rename('In-State')],
            axis=1).to_excel(writer, sheet_name='County Level Descriptive Stats', startrow=11, startcol=0)
writer.sheets['County Level Descriptive Stats'].conditional_format(11+2, 1, 11+9, 2, {'type':'no_blanks','format':currency_format})
writer.sheets['County Level Descriptive Stats'].write_string(11, 0, 'Dollars Approved', header_format)
# Next for Jobs Reported:
pd.concat([county_totals['Jobs Reported'].describe().rename('National'),
            county_totals[county_totals['State']==target_state]['Jobs Reported'].describe().rename('In-State')],
            axis=1).to_excel(writer, sheet_name='County Level Descriptive Stats', startrow=22, startcol=0)
writer.sheets['County Level Descriptive Stats'].conditional_format(22+2, 1, 22+9, 2, {'type':'no_blanks','format':number_format})
writer.sheets['County Level Descriptive Stats'].write_string(22, 0, 'Jobs Reported', header_format)
# Now for Charge-Offs
pd.concat([county_totals['Charge-Offs'].describe().rename('National'),
            county_totals[county_totals['State']==target_state]['Charge-Offs'].describe().rename('In-State')],
            axis=1).to_excel(writer, sheet_name='County Level Descriptive Stats', startrow=33, startcol=0)
writer.sheets['County Level Descriptive Stats'].conditional_format(33+2, 1, 33+9, 2, {'type':'no_blanks','format':number_format})
writer.sheets['County Level Descriptive Stats'].write_string(33, 0, 'Charge-Offs', header_format)
# Lastly Charge-Off Rate:
pd.concat([county_totals['Charge-Off Rate'].describe().rename('National'),
            county_totals[county_totals['State']==target_state]['Charge-Off Rate'].describe().rename('In-State')],
            axis=1).to_excel(writer, sheet_name='County Level Descriptive Stats', startrow=44, startcol=0)
writer.sheets['County Level Descriptive Stats'].conditional_format(44+2, 1, 44+9, 2, {'type':'no_blanks','format':percent_format})
writer.sheets['County Level Descriptive Stats'].write_string(44, 0, 'Charge-Off Rate', header_format)
# We include key stats for the target county.
key_facts.loc[['Number of Loans', 'Dollars Approved','Jobs Reported', 'Charge-Offs', 
            'Charge-Off Rate']][target_county+', '+target_state].to_excel(writer, sheet_name='County Level Descriptive Stats', startrow=0, startcol=5)
# We will format these appropriately.
writer.sheets['County Level Descriptive Stats'].conditional_format(1, 6, 1, 6, {'type':'no_blanks','format':number_format})
writer.sheets['County Level Descriptive Stats'].conditional_format(2, 6, 2, 6, {'type':'no_blanks','format':currency_format})
writer.sheets['County Level Descriptive Stats'].conditional_format(3, 6, 3, 6, {'type':'no_blanks','format':number_format})
writer.sheets['County Level Descriptive Stats'].conditional_format(4, 6, 4, 6, {'type':'no_blanks','format':number_format})
writer.sheets['County Level Descriptive Stats'].conditional_format(5, 6, 5, 6, {'type':'no_blanks','format':percent_format})
# Finally we make sure that each column has plenty of space.
writer.sheets['County Level Descriptive Stats'].set_column(0,2,20)
writer.sheets['County Level Descriptive Stats'].set_column(5,6,20)


# We can now move on to the second loan-level sheet.
loan_level_dollars.to_excel(writer, sheet_name='Loan Level Descriptive Stats', startrow=0, startcol=0)
# Now we format.
writer.sheets['Loan Level Descriptive Stats'].set_column('A:D',20,number_format)
writer.sheets['Loan Level Descriptive Stats'].conditional_format(2, 1, 0+9, 3, {'type':'no_blanks','format':currency_format})
# Add a title.
writer.sheets['Loan Level Descriptive Stats'].write_string(0, 0, 'Loan Dollars', header_format)
# And now we repeat for jobs reported
loan_level_jobs.to_excel(writer, sheet_name='Loan Level Descriptive Stats', startrow=11, startcol=0)
writer.sheets['Loan Level Descriptive Stats'].write_string(11, 0, 'Jobs Reported', header_format) 
  



########################################################################################################
# Demographics:
########################################################################################################
gender_df[columns_to_save].to_excel(writer, sheet_name='Demographics', startrow=0, startcol=1)
writer.sheets['Demographics'].write_string(0, 0, 'Gender', header_format) 
race_df[columns_to_save].to_excel(writer, sheet_name='Demographics', startrow=6, startcol=0)


ethnicity_df[columns_to_save].to_excel(writer, sheet_name='Demographics', startrow=45, startcol=0)
 


writer.sheets['Demographics'].set_column('A:N',20,number_format)
writer.sheets['Demographics'].set_column('A:A',25)
writer.sheets['Demographics'].set_column('D:D',20,percent_format)
writer.sheets['Demographics'].set_column('E:E',20,currency_format)
writer.sheets['Demographics'].set_column('F:F',20,percent_format)
writer.sheets['Demographics'].set_column('H:H',20,percent_format)
writer.sheets['Demographics'].set_column('I:I',20,currency_format)
writer.sheets['Demographics'].set_column('J:J',20,percent_format)
writer.sheets['Demographics'].set_column('L:L',20,percent_format)
writer.sheets['Demographics'].set_column('M:M',20,currency_format)
writer.sheets['Demographics'].set_column('N:N',20,percent_format)


########################################################################################################
# NAICs and Business:
########################################################################################################
job_bins_df[columns_to_save].to_excel(writer, sheet_name='Business Types', startrow=0, startcol=0)
writer.sheets['Business Types'].write_string(0, 0, 'Jobs Reported', header_format) 
naics_sector_df[columns_to_save].to_excel(writer, sheet_name='Business Types', startrow=8, startcol=0)
writer.sheets['Business Types'].write_string(8, 0, 'NAICs Sector', header_format) 

writer.sheets['Business Types'].write_string(33, 0, 'Top 5 NAICs Codes (by most loans)', header_format) 
naics_code_df.nlargest(n=5,columns='{} Loans'.format(target_county))[[
        '{} Loans'.format(target_county),
        '{} Loan Percent'.format(target_county),
        '{} Loan Dollars'.format(target_county),
        '{} Loan Dollars Percent'.format(target_county),
        '{} Loans'.format('Rest of State'),
        '{} Loan Percent'.format('Rest of State'),
        '{} Loan Dollars'.format('Rest of State'),
        '{} Loan Dollars Percent'.format('Rest of State')
        ]].to_excel(writer, sheet_name='Business Types', startrow=34, startcol=0)
writer.sheets['Business Types'].write_string(34, 0, 'NAICs Code', header_format) 

writer.sheets['Business Types'].write_string(42, 0, 'Top 5 NAICs Codes (by most loan dollars)', header_format) 
naics_code_df.nlargest(n=5,columns='{} Loan Dollars'.format(target_county))[[
        '{} Loans'.format(target_county),
        '{} Loan Percent'.format(target_county),
        '{} Loan Dollars'.format(target_county),
        '{} Loan Dollars Percent'.format(target_county),
        '{} Loans'.format('Rest of State'),
        '{} Loan Percent'.format('Rest of State'),
        '{} Loan Dollars'.format('Rest of State'),
        '{} Loan Dollars Percent'.format('Rest of State')
        ]].to_excel(writer, sheet_name='Business Types', startrow=43, startcol=0)
writer.sheets['Business Types'].write_string(43, 0, 'NAICs Code', header_format) 

writer.sheets['Business Types'].set_column('A:N',20,number_format)
writer.sheets['Business Types'].set_column('A:A',25)
writer.sheets['Business Types'].set_column('C:C',20,percent_format)
writer.sheets['Business Types'].set_column('D:D',20,currency_format)
writer.sheets['Business Types'].set_column('E:E',20,percent_format)
writer.sheets['Business Types'].set_column('G:G',20,percent_format)
writer.sheets['Business Types'].set_column('H:H',20,currency_format)
writer.sheets['Business Types'].set_column('I:I',20,percent_format)
writer.sheets['Business Types'].set_column('K:K',20,percent_format)
writer.sheets['Business Types'].set_column('L:L',20,currency_format)
writer.sheets['Business Types'].set_column('M:M',20,percent_format)


########################################################################################################
# Outliers (by loans):
########################################################################################################
race_df.drop('Unanswered',axis=0,level=0).drop('Unanswered',axis=0,level=1).nlargest(n=5,columns='Loan Outliers')[columns_to_save].to_excel(
    writer, sheet_name='Outliers (by loans)', startrow=1,startcol=0)
writer.sheets['Outliers (by loans)'].write_string(0, 0, 'Top 5 (by race/gender)', header_format) 

ethnicity_df.drop('Unknown/NotStated',axis=0,level=0).drop('Unanswered',axis=0,level=1).nlargest(n=3,columns='Loan Outliers')[columns_to_save].to_excel(
    writer, sheet_name='Outliers (by loans)', startrow=10,startcol=0)
writer.sheets['Outliers (by loans)'].write_string(9, 0, 'Top 3 (by ethnicity/gender)', header_format) 

job_bins_df.nlargest(n=2,columns='Loan Outliers')[columns_to_save].to_excel(
    writer, sheet_name='Outliers (by loans)', startrow=17,startcol=1)
writer.sheets['Outliers (by loans)'].write_string(16, 0, 'Top 2 (by jobs reported)', header_format) 
writer.sheets['Outliers (by loans)'].write_string(17, 0, 'Jobs Reported', header_format) 
writer.sheets['Outliers (by loans)'].write_string(17, 1, 'Jobs Reported', header_format) 


naics_sector_df.nlargest(n=5,columns='Loan Outliers')[columns_to_save].to_excel(
    writer, sheet_name='Outliers (by loans)', startrow=23,startcol=1)
writer.sheets['Outliers (by loans)'].write_string(22, 0, 'Top 5 (by NAICs sector)', header_format) 
writer.sheets['Outliers (by loans)'].write_string(23, 0, 'NAICs Sector', header_format) 
writer.sheets['Outliers (by loans)'].write_string(23, 1, 'NAICs Sector', header_format) 

naics_code_df.nlargest(n=5,columns='Loan Outliers')[[
        '{} Loans'.format(target_county),
        '{} Loan Percent'.format(target_county),
        '{} Loan Dollars'.format(target_county),
        '{} Loan Dollars Percent'.format(target_county),
        '{} Loans'.format('Rest of State'),
        '{} Loan Percent'.format('Rest of State'),
        '{} Loan Dollars'.format('Rest of State'),
        '{} Loan Dollars Percent'.format('Rest of State')
        ]].to_excel(writer, sheet_name='Outliers (by loans)', startrow=31, startcol=1)
writer.sheets['Outliers (by loans)'].write_string(31, 0, 'Top 5 (by NAICs code)', header_format) 
writer.sheets['Outliers (by loans)'].write_string(32, 0, 'NAICs Code', header_format) 
writer.sheets['Outliers (by loans)'].write_string(32, 1, 'NAICs Code', header_format) 

writer.sheets['Outliers (by loans)'].set_column('A:N',20,number_format)
writer.sheets['Outliers (by loans)'].set_column('A:B',25)
writer.sheets['Outliers (by loans)'].set_column('D:D',20,percent_format)
writer.sheets['Outliers (by loans)'].set_column('E:E',20,currency_format)
writer.sheets['Outliers (by loans)'].set_column('F:F',20,percent_format)
writer.sheets['Outliers (by loans)'].set_column('H:H',20,percent_format)
writer.sheets['Outliers (by loans)'].set_column('I:I',20,currency_format)
writer.sheets['Outliers (by loans)'].set_column('J:J',20,percent_format)
writer.sheets['Outliers (by loans)'].set_column('L:L',20,percent_format)
writer.sheets['Outliers (by loans)'].set_column('M:M',20,currency_format)
writer.sheets['Outliers (by loans)'].set_column('N:N',20,percent_format)

########################################################################################################
# Outliers (by loan dollars):
########################################################################################################
race_df.drop('Unanswered',axis=0,level=0).drop('Unanswered',axis=0,level=1).nlargest(n=5,columns='Loan Dollars Outliers')[columns_to_save].to_excel(
    writer, sheet_name='Outliers (by loan dollars)', startrow=1,startcol=0)
writer.sheets['Outliers (by loan dollars)'].write_string(0, 0, 'Top 5 (by race/gender)', header_format) 

ethnicity_df.drop('Unknown/NotStated',axis=0,level=0).drop('Unanswered',axis=0,level=1).nlargest(n=3,columns='Loan Dollars Outliers')[columns_to_save].to_excel(
    writer, sheet_name='Outliers (by loan dollars)', startrow=10,startcol=0)
writer.sheets['Outliers (by loan dollars)'].write_string(9, 0, 'Top 3 (by ethnicity/gender)', header_format) 

job_bins_df.nlargest(n=2,columns='Loan Dollars Outliers')[columns_to_save].to_excel(
    writer, sheet_name='Outliers (by loan dollars)', startrow=17,startcol=1)
writer.sheets['Outliers (by loan dollars)'].write_string(16, 0, 'Top 2 (by jobs reported)', header_format) 
writer.sheets['Outliers (by loan dollars)'].write_string(17, 0, 'Jobs Reported', header_format) 
writer.sheets['Outliers (by loan dollars)'].write_string(17, 1, 'Jobs Reported', header_format) 


naics_sector_df.nlargest(n=5,columns='Loan Dollars Outliers')[columns_to_save].to_excel(
    writer, sheet_name='Outliers (by loan dollars)', startrow=23,startcol=1)
writer.sheets['Outliers (by loan dollars)'].write_string(22, 0, 'Top 5 (by NAICs sector)', header_format) 
writer.sheets['Outliers (by loan dollars)'].write_string(23, 0, 'NAICs Sector', header_format) 
writer.sheets['Outliers (by loan dollars)'].write_string(23, 1, 'NAICs Sector', header_format) 

naics_code_df.nlargest(n=5,columns='Loan Dollars Outliers')[[
        '{} Loans'.format(target_county),
        '{} Loan Percent'.format(target_county),
        '{} Loan Dollars'.format(target_county),
        '{} Loan Dollars Percent'.format(target_county),
        '{} Loans'.format('Rest of State'),
        '{} Loan Percent'.format('Rest of State'),
        '{} Loan Dollars'.format('Rest of State'),
        '{} Loan Dollars Percent'.format('Rest of State')
        ]].to_excel(writer, sheet_name='Outliers (by loan dollars)', startrow=31, startcol=1)
writer.sheets['Outliers (by loan dollars)'].write_string(31, 0, 'Top 5 (by NAICs code)', header_format) 
writer.sheets['Outliers (by loan dollars)'].write_string(32, 0, 'NAICs Code', header_format) 
writer.sheets['Outliers (by loan dollars)'].write_string(32, 1, 'NAICs Code', header_format) 

writer.sheets['Outliers (by loan dollars)'].set_column('A:N',20,number_format)
writer.sheets['Outliers (by loan dollars)'].set_column('A:B',25)
writer.sheets['Outliers (by loan dollars)'].set_column('D:D',20,percent_format)
writer.sheets['Outliers (by loan dollars)'].set_column('E:E',20,currency_format)
writer.sheets['Outliers (by loan dollars)'].set_column('F:F',20,percent_format)
writer.sheets['Outliers (by loan dollars)'].set_column('H:H',20,percent_format)
writer.sheets['Outliers (by loan dollars)'].set_column('I:I',20,currency_format)
writer.sheets['Outliers (by loan dollars)'].set_column('J:J',20,percent_format)
writer.sheets['Outliers (by loan dollars)'].set_column('L:L',20,percent_format)
writer.sheets['Outliers (by loan dollars)'].set_column('M:M',20,currency_format)
writer.sheets['Outliers (by loan dollars)'].set_column('N:N',20,percent_format)

########################################################################################################
# Noting Time and closing the excel sheet writer.
########################################################################################################
stop=time.time()
writer.sheets['Overview'].write_string(0,
                                       6,
                                       'Time to Generate Report: {}m, {}s'.format(int((stop-start)/60),int((stop-start)%60)),
                                        header_format)
writer.sheets['Overview'].set_column('G:G',30,header_format) 
writer.close()